# [Lab5] SageMaker ML 파이프라인 구축

이 노트북에서는 SageMaker Pipelines를 사용하여 엔드투엔드 ML 워크플로우를 자동화합니다.

## 주요 내용
- SageMaker Pipelines 개념 및 구성 요소
- 데이터 전처리 파이프라인 단계
- 하이퍼파라미터 튜닝 단계
- 모델 평가 및 조건부 등록
- 파이프라인 실행 및 모니터링
- CI/CD 워크플로우 구축

## 1. 환경 설정 및 변수 로드

In [ ]:
# 이전 노트북에서 저장한 변수들 로드
%store -r

print("✅ 저장된 변수들을 로드했습니다.")
try:
    print(f"   - 실험 이름: {experiment_name}")
    print(f"   - 등록된 모델: {registered_model_name}")
    print(f"   - 최고 성능 모델: {best_model_name if 'best_model_name' in locals() else 'N/A'}")
except NameError:
    print("   일부 변수가 없습니다. 기본값을 사용합니다.")

In [ ]:
# 필수 라이브러리 임포트
import boto3
import pandas as pd
import sagemaker
import json
import os
from time import gmtime, strftime
from sagemaker.workflow.pipeline_context import PipelineSession
from sagemaker_studio import Project

print("✅ 라이브러리 임포트 완료")

## 2. 프로젝트 및 파이프라인 설정

SageMaker AI 프로젝트와 파이프라인 세션을 초기화합니다.

In [ ]:
# 프로젝트 및 파이프라인 설정
project = Project()
role = project.iam_role

# 세션 초기화
sagemaker_session = sagemaker.session.Session()
pipeline_session = PipelineSession()
region = sagemaker_session.boto_region_name
default_bucket = sagemaker_session.default_bucket()

# 파이프라인 설정
pipeline_name = f"bank-marketing-ml-pipeline"
model_package_group_name = f"BankMarketingModelPackageGroup"
base_job_prefix = "bank-marketing"

print(f"✅ 파이프라인 설정 완료")
print(f"   - 파이프라인 이름: {pipeline_name}")
print(f"   - 모델 패키지 그룹: {model_package_group_name}")
print(f"   - 기본 버킷: {default_bucket}")
print(f"   - 리전: {region}")
print(f"   - IAM 역할: {role}")

In [ ]:
# 파이프라인 스크립트 디렉토리 생성
!mkdir -p pipeline_scripts

print("✅ 파이프라인 스크립트 디렉토리 생성 완료")

## 3. 파이프라인 파라미터 정의

파이프라인 실행 시 조정 가능한 파라미터들을 정의합니다.

In [ ]:
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
    ParameterFloat
)

# 파이프라인 파라미터 정의
processing_instance_count = ParameterInteger(
    name="ProcessingInstanceCount", 
    default_value=1
)

processing_instance_type = ParameterString(
    name="ProcessingInstanceType", 
    default_value="ml.m5.xlarge"
)

training_instance_type = ParameterString(
    name="TrainingInstanceType", 
    default_value="ml.m5.xlarge"
)

model_approval_status = ParameterString(
    name="ModelApprovalStatus", 
    default_value="PendingManualApproval"
)

# 성능 임계값
auc_score_threshold = ParameterFloat(
    name="AucScoreThreshold",
    default_value=0.75
)

# 입력 데이터 경로
input_data_uri = ParameterString(
    name="InputDataUri",
    default_value=f"s3://{default_bucket}/bank-additional/bank-additional-full.csv"
)

print("✅ 파이프라인 파라미터 정의 완료")
print(f"   - 처리 인스턴스: {processing_instance_type.default_value}")
print(f"   - 훈련 인스턴스: {training_instance_type.default_value}")
print(f"   - AUC 임계값: {auc_score_threshold.default_value}")
print(f"   - 모델 승인 상태: {model_approval_status.default_value}")

## 4. 데이터 전처리 단계

은행 마케팅 데이터를 전처리하는 파이프라인 단계를 정의합니다.

In [ ]:
%%writefile "pipeline_scripts/bank_preprocess.py"

import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import argparse

def preprocess_data():
    """은행 마케팅 데이터 전처리 함수"""
    
    # 데이터 로드
    base_dir = "/opt/ml/processing"
    input_path = f"{base_dir}/input/bank-additional-full.csv"
    
    print(f"데이터 로드 중: {input_path}")
    df = pd.read_csv(input_path, sep=';')
    
    print(f"원본 데이터 형태: {df.shape}")
    print(f"타겟 분포: {df['y'].value_counts()}")
    
    # 타겟 변수 인코딩 (yes=1, no=0)
    df['y'] = df['y'].map({'yes': 1, 'no': 0})
    
    # 범주형 변수 원-핫 인코딩
    categorical_columns = [
        'job', 'marital', 'education', 'default', 'housing', 'loan',
        'contact', 'month', 'day_of_week', 'poutcome'
    ]
    
    print(f"범주형 변수 인코딩: {categorical_columns}")
    df_encoded = pd.get_dummies(df, columns=categorical_columns, drop_first=True)
    
    print(f"인코딩 후 데이터 형태: {df_encoded.shape}")
    
    # 특성과 타겟 분리
    X = df_encoded.drop('y', axis=1)
    y = df_encoded['y']
    
    # 데이터 분할 (70% 훈련, 15% 검증, 15% 테스트)
    X_temp, X_test, y_temp, y_test = train_test_split(
        X, y, test_size=0.15, random_state=42, stratify=y
    )
    
    X_train, X_val, y_train, y_val = train_test_split(
        X_temp, y_temp, test_size=0.176, random_state=42, stratify=y_temp  # 0.176 ≈ 0.15/0.85
    )
    
    print(f"훈련 데이터: {X_train.shape[0]} 샘플")
    print(f"검증 데이터: {X_val.shape[0]} 샘플")
    print(f"테스트 데이터: {X_test.shape[0]} 샘플")
    
    # XGBoost 형식으로 저장 (타겟이 첫 번째 열)
    train_data = pd.concat([y_train, X_train], axis=1)
    val_data = pd.concat([y_val, X_val], axis=1)
    test_data = pd.concat([y_test, X_test], axis=1)
    
    # CSV 파일로 저장
    train_data.to_csv(f"{base_dir}/train/train.csv", header=False, index=False)
    val_data.to_csv(f"{base_dir}/validation/validation.csv", header=False, index=False)
    test_data.to_csv(f"{base_dir}/test/test.csv", header=False, index=False)
    
    print("✅ 데이터 전처리 완료")
    print(f"   - 훈련 데이터 저장: {base_dir}/train/train.csv")
    print(f"   - 검증 데이터 저장: {base_dir}/validation/validation.csv")
    print(f"   - 테스트 데이터 저장: {base_dir}/test/test.csv")

if __name__ == "__main__":
    preprocess_data()

In [ ]:
# 데이터 전처리 단계 정의
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep

# SKLearn 프로세서 생성
sklearn_processor = SKLearnProcessor(
    framework_version="1.0-1",
    instance_type=processing_instance_type,
    instance_count=processing_instance_count,
    base_job_name=f"{base_job_prefix}-preprocess",
    role=role,
    sagemaker_session=pipeline_session,
)

# 전처리 단계 인수 정의
processor_args = sklearn_processor.run(
    inputs=[
        ProcessingInput(
            source=input_data_uri,
            destination="/opt/ml/processing/input"
        ),
    ],
    outputs=[
        ProcessingOutput(
            output_name="train",
            source="/opt/ml/processing/train",
            destination=f"s3://{default_bucket}/{base_job_prefix}/data/train"
        ),
        ProcessingOutput(
            output_name="validation",
            source="/opt/ml/processing/validation",
            destination=f"s3://{default_bucket}/{base_job_prefix}/data/validation"
        ),
        ProcessingOutput(
            output_name="test",
            source="/opt/ml/processing/test",
            destination=f"s3://{default_bucket}/{base_job_prefix}/data/test"
        )
    ],
    code="pipeline_scripts/bank_preprocess.py",
)

# 전처리 단계 생성
step_process = ProcessingStep(
    name="BankMarketingDataProcess",
    step_args=processor_args
)

print("✅ 데이터 전처리 단계 정의 완료")
print(f"   - 프로세서: SKLearnProcessor")
print(f"   - 인스턴스 타입: {processing_instance_type.default_value}")
print(f"   - 출력 경로: s3://{default_bucket}/{base_job_prefix}/data/")

## 5. 하이퍼파라미터 튜닝 단계

XGBoost 모델의 하이퍼파라미터를 자동으로 튜닝하는 단계를 정의합니다.

In [ ]:
from sagemaker.estimator import Estimator
from sagemaker.inputs import TrainingInput
from sagemaker.tuner import (
    IntegerParameter,
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner,
)
from sagemaker.workflow.steps import TuningStep

# XGBoost 컨테이너 이미지 URI
image_uri = sagemaker.image_uris.retrieve(
    framework="xgboost",
    region=region,
    version="1.7-1",
    py_version="py3",
    instance_type=training_instance_type,
)

# 고정 하이퍼파라미터
fixed_hyperparameters = {
    "eval_metric": "auc",
    "objective": "binary:logistic",
    "num_round": "100",
    "early_stopping_rounds": "10",
    "verbosity": "1"
}

# XGBoost Estimator 생성
xgb_estimator = Estimator(
    image_uri=image_uri,
    instance_type=training_instance_type,
    instance_count=1,
    hyperparameters=fixed_hyperparameters,
    output_path=f"s3://{default_bucket}/{base_job_prefix}/models",
    base_job_name=f"{base_job_prefix}-train",
    sagemaker_session=pipeline_session,
    role=role,
)

print(f"✅ XGBoost Estimator 생성 완료")
print(f"   - 이미지 URI: {image_uri}")
print(f"   - 인스턴스 타입: {training_instance_type.default_value}")
print(f"   - 고정 하이퍼파라미터: {fixed_hyperparameters}")

In [ ]:
# 하이퍼파라미터 튜닝 범위 정의
hyperparameter_ranges = {
    "eta": ContinuousParameter(0.01, 0.3),           # 학습률
    "max_depth": IntegerParameter(3, 10),            # 트리 최대 깊이
    "min_child_weight": ContinuousParameter(1, 10),  # 최소 자식 가중치
    "subsample": ContinuousParameter(0.7, 1.0),      # 서브샘플링 비율
    "colsample_bytree": ContinuousParameter(0.7, 1.0), # 특성 서브샘플링
    "alpha": ContinuousParameter(0, 2),              # L1 정규화
    "lambda": ContinuousParameter(1, 2),             # L2 정규화
}

# 목적 메트릭
objective_metric_name = "validation:auc"

# 하이퍼파라미터 튜너 생성
tuner = HyperparameterTuner(
    xgb_estimator,
    objective_metric_name,
    hyperparameter_ranges,
    max_jobs=6,        # 최대 튜닝 작업 수
    max_parallel_jobs=2,  # 병렬 실행 작업 수
    objective_type="Maximize",
    base_tuning_job_name=f"{base_job_prefix}-tuning"
)

print(f"✅ 하이퍼파라미터 튜너 생성 완료")
print(f"   - 목적 메트릭: {objective_metric_name}")
print(f"   - 최대 작업 수: 6개")
print(f"   - 병렬 작업 수: 2개")
print(f"   - 튜닝 파라미터: {list(hyperparameter_ranges.keys())}")

In [ ]:
# 하이퍼파라미터 튜닝 단계 생성
hpo_args = tuner.fit(
    inputs={
        "train": TrainingInput(
            s3_data=step_process.properties.ProcessingOutputConfig.Outputs["train"].S3Output.S3Uri,
            content_type="text/csv",
        ),
        "validation": TrainingInput(
            s3_data=step_process.properties.ProcessingOutputConfig.Outputs["validation"].S3Output.S3Uri,
            content_type="text/csv",
        ),
    }
)

step_tuning = TuningStep(
    name="BankMarketingHyperParameterTuning",
    step_args=hpo_args,
)

print("✅ 하이퍼파라미터 튜닝 단계 생성 완료")
print(f"   - 단계 이름: BankMarketingHyperParameterTuning")
print(f"   - 입력 데이터: 전처리 단계의 train/validation 출력")

## 6. 모델 평가 단계

최적 모델의 성능을 테스트 데이터로 평가하는 단계를 정의합니다.

In [ ]:
%%writefile "pipeline_scripts/bank_evaluate.py"

import json
import pathlib
import pickle
import tarfile
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.metrics import roc_auc_score, accuracy_score, classification_report, confusion_matrix

def evaluate_model():
    """모델 평가 함수"""
    
    print("🔍 모델 평가 시작")
    
    # 모델 로드
    model_path = "/opt/ml/processing/model/model.tar.gz"
    print(f"모델 로드 중: {model_path}")
    
    with tarfile.open(model_path) as tar:
        tar.extractall(path=".")
    
    model = pickle.load(open("xgboost-model", "rb"))
    print("✅ 모델 로드 완료")
    
    # 테스트 데이터 로드
    test_path = "/opt/ml/processing/test/test.csv"
    print(f"테스트 데이터 로드 중: {test_path}")
    
    df = pd.read_csv(test_path, header=None)
    y_test = df.iloc[:, 0].to_numpy()
    X_test = df.iloc[:, 1:].values
    
    print(f"테스트 데이터 형태: {X_test.shape}")
    print(f"테스트 레이블 분포: {np.bincount(y_test)}")
    
    # XGBoost DMatrix 생성
    dtest = xgb.DMatrix(X_test)
    
    # 예측 수행
    print("📊 예측 수행 중...")
    predictions_proba = model.predict(dtest)
    predictions_binary = (predictions_proba > 0.5).astype(int)
    
    # 성능 메트릭 계산
    auc_score = roc_auc_score(y_test, predictions_proba)
    accuracy = accuracy_score(y_test, predictions_binary)
    
    # 혼동 행렬
    cm = confusion_matrix(y_test, predictions_binary)
    tn, fp, fn, tp = cm.ravel()
    
    # 추가 메트릭 계산
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    print(f"\n📊 평가 결과:")
    print(f"   - AUC: {auc_score:.4f}")
    print(f"   - 정확도: {accuracy:.4f}")
    print(f"   - 정밀도: {precision:.4f}")
    print(f"   - 재현율: {recall:.4f}")
    print(f"   - F1 점수: {f1_score:.4f}")
    
    # 평가 리포트 생성
    report_dict = {
        "classification_metrics": {
            "auc_score": {
                "value": float(auc_score),
                "standard_deviation": 0.0
            },
            "accuracy": {
                "value": float(accuracy),
                "standard_deviation": 0.0
            },
            "precision": {
                "value": float(precision),
                "standard_deviation": 0.0
            },
            "recall": {
                "value": float(recall),
                "standard_deviation": 0.0
            },
            "f1_score": {
                "value": float(f1_score),
                "standard_deviation": 0.0
            }
        },
        "confusion_matrix": {
            "true_negative": int(tn),
            "false_positive": int(fp),
            "false_negative": int(fn),
            "true_positive": int(tp)
        }
    }
    
    # 평가 결과 저장
    output_dir = "/opt/ml/processing/evaluation"
    pathlib.Path(output_dir).mkdir(parents=True, exist_ok=True)
    
    evaluation_path = f"{output_dir}/evaluation.json"
    with open(evaluation_path, "w") as f:
        f.write(json.dumps(report_dict, indent=2))
    
    print(f"✅ 평가 결과 저장: {evaluation_path}")
    
    return report_dict

if __name__ == "__main__":
    evaluate_model()

In [ ]:
# 모델 평가 단계 정의
from sagemaker.processing import ScriptProcessor
from sagemaker.workflow.properties import PropertyFile

# 스크립트 프로세서 생성
script_eval = ScriptProcessor(
    image_uri=image_uri,
    command=["python3"],
    instance_type=processing_instance_type,
    instance_count=1,
    base_job_name=f"{base_job_prefix}-eval",
    role=role,
    sagemaker_session=pipeline_session,
)

# 평가 단계 인수 정의
eval_args = script_eval.run(
    inputs=[
        ProcessingInput(
            source=step_tuning.get_top_model_s3_uri(
                top_k=0, 
                s3_bucket=default_bucket, 
                prefix=f"{base_job_prefix}/models"
            ),
            destination="/opt/ml/processing/model"
        ),
        ProcessingInput(
            source=step_process.properties.ProcessingOutputConfig.Outputs["test"].S3Output.S3Uri,
            destination="/opt/ml/processing/test"
        )
    ],
    outputs=[
        ProcessingOutput(
            output_name="evaluation",
            source="/opt/ml/processing/evaluation",
            destination=f"s3://{default_bucket}/{base_job_prefix}/evaluation"
        ),
    ],
    code="pipeline_scripts/bank_evaluate.py",
)

# 평가 리포트 속성 파일
evaluation_report = PropertyFile(
    name="BankMarketingEvaluationReport",
    output_name="evaluation",
    path="evaluation.json"
)

# 평가 단계 생성
step_eval = ProcessingStep(
    name="BankMarketingEvalModel",
    step_args=eval_args,
    property_files=[evaluation_report],
)

print("✅ 모델 평가 단계 생성 완료")
print(f"   - 단계 이름: BankMarketingEvalModel")
print(f"   - 입력: 최적 모델 + 테스트 데이터")
print(f"   - 출력: 평가 리포트 (evaluation.json)")

## 7. 모델 등록 단계

성능이 임계값을 넘는 모델을 Model Registry에 등록하는 단계를 정의합니다.

In [ ]:
from sagemaker import Model
from sagemaker.workflow.model_step import ModelStep
from sagemaker.model_metrics import MetricsSource, ModelMetrics

# 모델 생성
model = Model(
    image_uri=image_uri,
    model_data=step_tuning.get_top_model_s3_uri(
        top_k=0, 
        s3_bucket=default_bucket, 
        prefix=f"{base_job_prefix}/models"
    ),
    sagemaker_session=pipeline_session,
    role=role,
)

# 모델 메트릭 정의
model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri="{}/evaluation.json".format(
            step_eval.arguments["ProcessingOutputConfig"]["Outputs"][0]["S3Output"]["S3Uri"]
        ),
        content_type="application/json",
    )
)

# 모델 등록 인수 정의
register_args = model.register(
    content_types=["text/csv"],
    response_types=["text/csv"],
    inference_instances=["ml.t2.medium", "ml.m5.large", "ml.m5.xlarge"],
    transform_instances=["ml.m5.xlarge"],
    model_package_group_name=model_package_group_name,
    approval_status=model_approval_status,
    model_metrics=model_metrics,
    description="은행 마케팅 캠페인 정기예금 가입 예측 모델",
    model_name="BankMarketingXGBoostModel"
)

# 모델 등록 단계 생성
step_register = ModelStep(
    name="BankMarketingRegisterModel",
    step_args=register_args
)

print("✅ 모델 등록 단계 생성 완료")
print(f"   - 모델 패키지 그룹: {model_package_group_name}")
print(f"   - 승인 상태: {model_approval_status.default_value}")
print(f"   - 지원 인스턴스: ml.t2.medium, ml.m5.large, ml.m5.xlarge")

## 8. 조건부 단계

모델 성능이 임계값을 넘을 때만 모델을 등록하는 조건부 로직을 정의합니다.

In [ ]:
from sagemaker.workflow.conditions import ConditionGreaterThan
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.workflow.functions import JsonGet

# AUC 점수 조건 정의
cond_gte = ConditionGreaterThan(
    left=JsonGet(
        step_name=step_eval.name,
        property_file=evaluation_report,
        json_path="classification_metrics.auc_score.value",
    ),
    right=auc_score_threshold,
)

# 조건부 단계 생성
step_cond = ConditionStep(
    name="CheckAUCScoreBankMarketingEvaluation",
    conditions=[cond_gte],
    if_steps=[step_register],  # AUC가 임계값보다 높으면 모델 등록
    else_steps=[],             # 그렇지 않으면 아무것도 하지 않음
)

print("✅ 조건부 단계 생성 완료")
print(f"   - 조건: AUC > {auc_score_threshold.default_value}")
print(f"   - True일 때: 모델 등록")
print(f"   - False일 때: 아무 작업 없음")

## 9. 파이프라인 생성 및 정의

모든 단계를 연결하여 완전한 ML 파이프라인을 생성합니다.

In [ ]:
from sagemaker.workflow.pipeline import Pipeline

# 파이프라인 생성
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        processing_instance_count,
        processing_instance_type,
        training_instance_type,
        model_approval_status,
        auc_score_threshold,
        input_data_uri,
    ],
    steps=[
        step_process,    # 1. 데이터 전처리
        step_tuning,     # 2. 하이퍼파라미터 튜닝
        step_eval,       # 3. 모델 평가
        step_cond,       # 4. 조건부 모델 등록
    ],
    sagemaker_session=pipeline_session,
)

print("✅ 파이프라인 생성 완료")
print(f"   - 파이프라인 이름: {pipeline_name}")
print(f"   - 총 단계 수: 4개")
print(f"   - 파라미터 수: 6개")

In [ ]:
# 파이프라인 정의 확인
definition = json.loads(pipeline.definition())

print("📋 파이프라인 정의 요약:")
print(f"   - 버전: {definition['Version']}")
print(f"   - 파라미터 수: {len(definition['Parameters'])}")
print(f"   - 단계 수: {len(definition['Steps'])}")

print(f"\n🔧 파라미터 목록:")
for param in definition['Parameters']:
    print(f"   - {param['Name']}: {param['Type']} (기본값: {param['DefaultValue']})")

print(f"\n📊 단계 목록:")
for step in definition['Steps']:
    print(f"   - {step['Name']}: {step['Type']}")

# 전체 정의를 파일로 저장 (선택사항)
with open('pipeline_definition.json', 'w') as f:
    json.dump(definition, f, indent=2)

print(f"\n💾 파이프라인 정의가 'pipeline_definition.json'에 저장되었습니다.")

## 10. 파이프라인 실행

정의된 파이프라인을 SageMaker에 등록하고 실행합니다.

In [ ]:
# 파이프라인 등록/업데이트
print("🔄 파이프라인 등록 중...")

try:
    pipeline.upsert(role_arn=role)
    print(f"✅ 파이프라인 등록 완료: {pipeline_name}")
    
    # 파이프라인 정보 출력
    print(f"\n📊 파이프라인 정보:")
    print(f"   - 이름: {pipeline.name}")
    print(f"   - ARN: {pipeline.describe()['PipelineArn']}")
    print(f"   - 상태: {pipeline.describe()['PipelineStatus']}")
    print(f"   - 생성 시간: {pipeline.describe()['CreationTime']}")
    
except Exception as e:
    print(f"❌ 파이프라인 등록 실패: {e}")
    print("IAM 권한이나 리소스 제한을 확인해주세요.")

In [ ]:
# 파이프라인 실행
print("🚀 파이프라인 실행 시작...")

try:
    execution = pipeline.start(
        parameters={
            "ProcessingInstanceType": "ml.m5.xlarge",
            "TrainingInstanceType": "ml.m5.xlarge",
            "AucScoreThreshold": 0.75,
            "ModelApprovalStatus": "PendingManualApproval"
        }
    )
    
    print(f"✅ 파이프라인 실행 시작됨")
    print(f"   - 실행 ARN: {execution.arn}")
    print(f"   - 실행 이름: {execution.arn.split('/')[-1]}")
    
    # 실행 상태 확인
    print(f"\n📊 실행 상태: {execution.describe()['PipelineExecutionStatus']}")
    print(f"   - 시작 시간: {execution.describe()['CreationTime']}")
    
    # 실행 모니터링 안내
    print(f"\n💡 실행 모니터링:")
    print(f"   - SageMaker Studio: 파이프라인 > {pipeline_name} > 실행")
    print(f"   - AWS 콘솔: SageMaker > 파이프라인 > {pipeline_name}")
    print(f"   - 예상 소요 시간: 20-30분")
    
except Exception as e:
    print(f"❌ 파이프라인 실행 실패: {e}")
    print("파라미터나 리소스 설정을 확인해주세요.")

## 11. 파이프라인 모니터링

실행 중인 파이프라인의 상태를 모니터링하고 결과를 확인합니다.

In [ ]:
# 파이프라인 실행 상태 확인
if 'execution' in locals():
    print(f"📊 파이프라인 실행 상태 확인: {execution.arn.split('/')[-1]}")
    
    try:
        # 현재 상태 조회
        status = execution.describe()
        
        print(f"\n🔍 실행 정보:")
        print(f"   - 상태: {status['PipelineExecutionStatus']}")
        print(f"   - 시작 시간: {status['CreationTime']}")
        
        if 'LastModifiedTime' in status:
            print(f"   - 마지막 수정: {status['LastModifiedTime']}")
        
        if status['PipelineExecutionStatus'] == 'Failed':
            print(f"   - 실패 이유: {status.get('FailureReason', 'N/A')}")
        
        # 단계별 상태 확인
        steps = execution.list_steps()
        
        print(f"\n📋 단계별 상태:")
        for step in steps:
            step_name = step['StepName']
            step_status = step['StepStatus']
            start_time = step.get('StartTime', 'N/A')
            
            status_emoji = {
                'Executing': '🔄',
                'Succeeded': '✅',
                'Failed': '❌',
                'Stopping': '⏸️',
                'Stopped': '⏹️'
            }.get(step_status, '❓')
            
            print(f"   {status_emoji} {step_name}: {step_status}")
            if start_time != 'N/A':
                print(f"      시작 시간: {start_time}")
            
            if step_status == 'Failed' and 'FailureReason' in step:
                print(f"      실패 이유: {step['FailureReason']}")
        
    except Exception as e:
        print(f"❌ 상태 조회 실패: {e}")
else:
    print("⚠️ 실행 중인 파이프라인이 없습니다.")
    print("위의 셀에서 파이프라인을 먼저 실행해주세요.")

In [ ]:
# 파이프라인 완료 대기 (선택사항)
# 주의: 이 셀은 파이프라인이 완료될 때까지 실행됩니다 (20-30분 소요)

WAIT_FOR_COMPLETION = False  # True로 변경하면 완료까지 대기

if WAIT_FOR_COMPLETION and 'execution' in locals():
    print("⏳ 파이프라인 완료 대기 중...")
    print("   이 과정은 20-30분 정도 소요될 수 있습니다.")
    print("   중단하려면 커널을 재시작하세요.")
    
    try:
        execution.wait(delay=60, max_attempts=60)  # 최대 60분 대기
        
        final_status = execution.describe()
        print(f"\n🎉 파이프라인 실행 완료!")
        print(f"   - 최종 상태: {final_status['PipelineExecutionStatus']}")
        print(f"   - 완료 시간: {final_status.get('LastModifiedTime', 'N/A')}")
        
        if final_status['PipelineExecutionStatus'] == 'Succeeded':
            print(f"   ✅ 모든 단계가 성공적으로 완료되었습니다!")
        else:
            print(f"   ⚠️ 파이프라인이 실패했습니다. 로그를 확인해주세요.")
            
    except Exception as e:
        print(f"❌ 대기 중 오류 발생: {e}")
        
else:
    print("💡 파이프라인 완료 대기를 건너뜁니다.")
    print("   완료 상태를 확인하려면 위의 모니터링 셀을 다시 실행하세요.")
    print("   또는 WAIT_FOR_COMPLETION을 True로 설정하고 이 셀을 다시 실행하세요.")

## 12. 파이프라인 관리

파이프라인의 실행 이력을 조회하고 관리하는 방법을 알아봅니다.

In [ ]:
# 파이프라인 실행 이력 조회
print(f"📋 파이프라인 실행 이력: {pipeline_name}")

try:
    executions = pipeline.list_executions()
    
    if executions:
        print(f"\n총 {len(executions)}개의 실행 기록이 있습니다:")
        
        for i, exec_summary in enumerate(executions[:5], 1):  # 최근 5개만 표시
            exec_name = exec_summary['PipelineExecutionArn'].split('/')[-1]
            status = exec_summary['PipelineExecutionStatus']
            start_time = exec_summary['StartTime']
            
            status_emoji = {
                'Executing': '🔄',
                'Succeeded': '✅',
                'Failed': '❌',
                'Stopping': '⏸️',
                'Stopped': '⏹️'
            }.get(status, '❓')
            
            print(f"   {i}. {status_emoji} {exec_name}")
            print(f"      상태: {status}")
            print(f"      시작: {start_time}")
            
            if 'PipelineExecutionDescription' in exec_summary:
                print(f"      설명: {exec_summary['PipelineExecutionDescription']}")
            print()
    else:
        print("   실행 기록이 없습니다.")
        
except Exception as e:
    print(f"❌ 실행 이력 조회 실패: {e}")

In [ ]:
# 모델 레지스트리 확인
print(f"🏪 모델 레지스트리 확인: {model_package_group_name}")

try:
    sm_client = boto3.client('sagemaker')
    
    # 모델 패키지 그룹 존재 확인
    try:
        group_info = sm_client.describe_model_package_group(
            ModelPackageGroupName=model_package_group_name
        )
        print(f"✅ 모델 패키지 그룹 존재: {model_package_group_name}")
        print(f"   - 생성 시간: {group_info['CreationTime']}")
        print(f"   - 상태: {group_info['ModelPackageGroupStatus']}")
        
        # 등록된 모델 패키지 조회
        packages = sm_client.list_model_packages(
            ModelPackageGroupName=model_package_group_name,
            SortBy='CreationTime',
            SortOrder='Descending'
        )
        
        if packages['ModelPackageSummaryList']:
            print(f"\n📦 등록된 모델 패키지 ({len(packages['ModelPackageSummaryList'])}개):")
            
            for i, package in enumerate(packages['ModelPackageSummaryList'][:3], 1):
                print(f"   {i}. {package['ModelPackageArn'].split('/')[-1]}")
                print(f"      상태: {package['ModelPackageStatus']}")
                print(f"      승인: {package['ModelApprovalStatus']}")
                print(f"      생성: {package['CreationTime']}")
                print()
        else:
            print("   등록된 모델 패키지가 없습니다.")
            print("   파이프라인이 성공적으로 완료되고 AUC 임계값을 넘어야 모델이 등록됩니다.")
            
    except sm_client.exceptions.ResourceNotFound:
        print(f"⚠️ 모델 패키지 그룹이 아직 생성되지 않았습니다: {model_package_group_name}")
        print("   파이프라인이 성공적으로 완료되면 자동으로 생성됩니다.")
        
except Exception as e:
    print(f"❌ 모델 레지스트리 확인 실패: {e}")

## 13. CI/CD 워크플로우 구축

파이프라인을 CI/CD 시스템과 통합하는 방법을 알아봅니다.

In [ ]:
# CI/CD 통합을 위한 파이프라인 트리거 함수
def trigger_pipeline_execution(
    pipeline_name,
    parameters=None,
    execution_display_name=None
):
    """파이프라인 실행을 트리거하는 함수"""
    
    try:
        # 기본 파라미터 설정
        if parameters is None:
            parameters = {
                "ProcessingInstanceType": "ml.m5.xlarge",
                "TrainingInstanceType": "ml.m5.xlarge",
                "AucScoreThreshold": 0.75,
                "ModelApprovalStatus": "PendingManualApproval"
            }
        
        # 실행 이름 생성
        if execution_display_name is None:
            timestamp = strftime('%Y%m%d-%H%M%S', gmtime())
            execution_display_name = f"pipeline-execution-{timestamp}"
        
        # 파이프라인 실행
        execution = pipeline.start(
            parameters=parameters,
            execution_display_name=execution_display_name
        )
        
        return {
            'success': True,
            'execution_arn': execution.arn,
            'execution_name': execution.arn.split('/')[-1],
            'message': f'파이프라인 실행 시작: {execution_display_name}'
        }
        
    except Exception as e:
        return {
            'success': False,
            'error': str(e),
            'message': f'파이프라인 실행 실패: {e}'
        }

# 예제 실행
print("🔧 CI/CD 통합 함수 정의 완료")
print("\n💡 사용 예시:")
print("```python")
print("result = trigger_pipeline_execution(")
print("    pipeline_name='bank-marketing-ml-pipeline',")
print("    parameters={'AucScoreThreshold': 0.8},")
print("    execution_display_name='production-deployment'")
print(")")
print("```")

In [ ]:
# 웹훅 스타일 API 엔드포인트 예제
def create_pipeline_webhook_handler():
    """Lambda 함수나 API Gateway에서 사용할 수 있는 웹훅 핸들러 예제"""
    
    webhook_code = '''
import json
import boto3
from datetime import datetime

def lambda_handler(event, context):
    """파이프라인 실행을 위한 Lambda 핸들러"""
    
    try:
        # SageMaker 클라이언트 생성
        sagemaker_client = boto3.client('sagemaker')
        
        # 요청 파라미터 파싱
        body = json.loads(event.get('body', '{}'))
        
        pipeline_name = body.get('pipeline_name', 'bank-marketing-ml-pipeline')
        parameters = body.get('parameters', {})
        
        # 기본 파라미터 설정
        default_params = {
            'ProcessingInstanceType': 'ml.m5.xlarge',
            'TrainingInstanceType': 'ml.m5.xlarge',
            'AucScoreThreshold': '0.75',
            'ModelApprovalStatus': 'PendingManualApproval'
        }
        
        # 파라미터 병합
        final_params = {**default_params, **parameters}
        
        # 파이프라인 실행
        response = sagemaker_client.start_pipeline_execution(
            PipelineName=pipeline_name,
            PipelineParameters=[
                {'Name': k, 'Value': str(v)} for k, v in final_params.items()
            ],
            PipelineExecutionDisplayName=f"webhook-{datetime.now().strftime('%Y%m%d-%H%M%S')}"
        )
        
        return {
            'statusCode': 200,
            'body': json.dumps({
                'success': True,
                'execution_arn': response['PipelineExecutionArn'],
                'message': 'Pipeline execution started successfully'
            })
        }
        
    except Exception as e:
        return {
            'statusCode': 500,
            'body': json.dumps({
                'success': False,
                'error': str(e),
                'message': 'Pipeline execution failed'
            })
        }
'''
    
    return webhook_code

# 웹훅 코드 생성 및 저장
webhook_code = create_pipeline_webhook_handler()

with open('pipeline_webhook.py', 'w') as f:
    f.write(webhook_code)

print("🌐 웹훅 핸들러 코드 생성 완료")
print("   - 파일: pipeline_webhook.py")
print("   - 용도: Lambda 함수나 API Gateway에서 사용")
print("   - 기능: HTTP 요청으로 파이프라인 실행 트리거")

## 14. 파이프라인 결과 요약

구축된 ML 파이프라인의 전체적인 결과를 요약합니다.

In [ ]:
# 파이프라인 구축 결과 요약
print("📋 SageMaker ML 파이프라인 구축 완료!")
print("=" * 60)

print(f"🏗️ 구축된 파이프라인:")
print(f"   - 이름: {pipeline_name}")
print(f"   - 모델 패키지 그룹: {model_package_group_name}")
print(f"   - 기본 버킷: {default_bucket}")

print(f"\n🔧 파이프라인 구성 요소:")
print(f"   1. 데이터 전처리 (SKLearnProcessor)")
print(f"      - 은행 마케팅 데이터 전처리")
print(f"      - 원-핫 인코딩 및 데이터 분할")
print(f"   2. 하이퍼파라미터 튜닝 (XGBoost)")
print(f"      - 6개 작업, 2개 병렬 실행")
print(f"      - AUC 최적화")
print(f"   3. 모델 평가 (테스트 데이터)")
print(f"      - AUC, 정확도, 정밀도, 재현율 계산")
print(f"   4. 조건부 모델 등록")
print(f"      - AUC > {auc_score_threshold.default_value}일 때만 등록")

print(f"\n⚙️ 파라미터 설정:")
print(f"   - 처리 인스턴스: {processing_instance_type.default_value}")
print(f"   - 훈련 인스턴스: {training_instance_type.default_value}")
print(f"   - AUC 임계값: {auc_score_threshold.default_value}")
print(f"   - 승인 상태: {model_approval_status.default_value}")

print(f"\n🚀 실행 방법:")
print(f"   - SageMaker Studio: 파이프라인 탭에서 실행")
print(f"   - 프로그래밍: pipeline.start() 메서드")
print(f"   - CI/CD: 웹훅 또는 Lambda 함수")

print(f"\n📊 모니터링:")
print(f"   - 실행 상태: execution.describe()")
print(f"   - 단계별 로그: CloudWatch Logs")
print(f"   - 모델 등록: Model Registry")

print(f"\n💡 다음 단계:")
print(f"   1. 파이프라인 실행 및 모니터링")
print(f"   2. 등록된 모델 승인 및 배포")
print(f"   3. CI/CD 시스템과 통합")
print(f"   4. 스케줄링 및 자동화 설정")

print("\n" + "=" * 60)
print("🎉 ML 파이프라인 구축 완료!")
print("이제 자동화된 ML 워크플로우를 통해 모델을 지속적으로 개선할 수 있습니다.")